In [12]:
#import streamlit as st
import pandas as pd
from mplsoccer import Sbopen,Pitch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost
from itertools import combinations_with_replacement
from sklearn.linear_model import LinearRegression
import joblib
from sklearn.model_selection import cross_val_score

In [2]:
parser = Sbopen()
team = "Barcelona"

In [5]:
df = pd.read_csv('laliga_events.csv')

/var/folders/dq/pg0mssxn6yzdwtcdrwnt0bs80000gn/T/ipykernel_86970/978393080.py:1: DtypeWarning: Columns (3,12,14,16,19,21,23,27,31,33,38,40,42,43,46,47,50,52,53,54,55,56,57,58,59,60,61,62,63,65,67,68,69,70,71,72,76,80,81,82,83,85,86,87,88,89,90,91,92,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('laliga_events.csv')


In [11]:
def getPossessionsPerMatchDf(df):
    unique_possessions = df[(df['type_name'] == 'Shot')]['possession'].unique()
    if len(unique_possessions)>0:
        df_filtered = pd.DataFrame()

        for possession in unique_possessions:
            first_row_index = df.loc[df['possession'] == possession].index[0]
            index_shot = df.loc[(df['possession'] == possession) & (df['type_name'] == 'Shot')].index
            if len(index_shot) > 0:
                df_copy = df.copy()
                df_selected = df_copy[first_row_index:index_shot[0].item() + 1]
                df_filtered = pd.concat([df_filtered, df_selected], ignore_index=True)

        df_filtered.reset_index(drop=True, inplace=True)


        df_filtered2 = df_filtered.copy()

        for possession in unique_possessions:
            possession_mask = df_filtered2['possession'] == possession
            try:
                shot_xg = df_filtered2.loc[possession_mask & (df_filtered2['type_name'] == 'Shot'), 'shot_statsbomb_xg'].values[0]
                df_filtered2.loc[possession_mask, 'xG'] = shot_xg
            except IndexError:
                print("no shots found in filtered dataframe")

        df2 = df.copy()
        df2 = df2[~df2['possession'].isin(unique_possessions)]
        df2 = pd.concat([df2, df_filtered2], ignore_index=True)

        df2['shot_end'] = df2['possession'].isin(unique_possessions).astype(int)
        return df2

In [ ]:
def featureEngineeringForModel(df):
    
    df3 = df.copy()

    #columns with coordinates
    df3["x0"] = df3['x']
    df3["c0"] = abs(40 - df['y']) 
    df3["x1"] = df3['end_x']
    df3["c1"] = abs(40 - df['end_y']) 


    #for plotting
    df3["y0"] = df3['y']
    df3["y1"] = df3['end_y']


    #model variables
    var = ["x0", "x1", "c0", "c1"]

    #combinations
    inputs = []
    #one variable combinations
    inputs.extend(combinations_with_replacement(var, 1))
    #2 variable combinations
    inputs.extend(combinations_with_replacement(var, 2))
    #3 variable combinations
    inputs.extend(combinations_with_replacement(var, 3))

    #make new columns
    for i in inputs:
        #columns length 1 already exist
        if len(i) > 1:
            #column name
            column = ''
            x = 1
            for c in i:
                #add column name to be x0x1c0 for example
                column += c
                #multiply values in column
                x = x*df3[c]
            #create a new column in df
            df3[column] = x
            #add column to model variables
            var.append(column)

    return df3,var

In [ ]:
def passingProbabilityModel(df,var):
    passes = df.loc[df["type_name"].isin(["Pass"])]
    X = passes[var].values 
    y = passes["shot_end"].values
    unique_classes, class_counts = np.unique(y, return_counts=True)
    # Check if all elements are greater than 2
    all_greater_than_two = all(count > 2 for count in class_counts)
    if all_greater_than_two:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 123, stratify = y)
        model = xgboost.XGBClassifier(n_estimators = 100, ccp_alpha=0, max_depth=4, min_samples_leaf=10,
                            random_state=123)
        #print(np.mean(scores), np.std(scores))
        model.fit(X_train, y_train)

    joblib.dump(model, 'xGBmodel.pkl')


In [ ]:
def shotProbabilityModel(df,var):
    

In [ ]:
def getPossessionChainsFromSelection(df):

    df2 = df.copy()
    #columns with coordinates
    df2["x0"] = df2['x']
    df2["c0"] = abs(40 - df2['y']) 
    df2["x1"] = df2['end_x']
    df2["c1"] = abs(40 - df2['end_y']) 


    #for plotting
    df2["y0"] = df2['y']
    df2["y1"] = df2['end_y']


    #model variables
    var = ["x0", "x1", "c0", "c1"]

    #combinations
    inputs = []
    #one variable combinations
    inputs.extend(combinations_with_replacement(var, 1))
    #2 variable combinations
    inputs.extend(combinations_with_replacement(var, 2))
    #3 variable combinations
    inputs.extend(combinations_with_replacement(var, 3))

    #make new columns
    for i in inputs:
        #columns length 1 already exist
        if len(i) > 1:
            #column name
            column = ''
            x = 1
            for c in i:
                #add column name to be x0x1c0 for example
                column += c
                #multiply values in column
                x = x*df2[c]
            #create a new column in df
            df2[column] = x
            #add column to model variables
            var.append(column)

    passes = df2.loc[df2["type_name"].isin(["Pass"])]

    return passes

In [13]:
def getShotProbability(df,var):
    try:
        shot_ended = df.loc[df["shot_end"] == 1]
        X2 = shot_ended[var].values
        y2 = shot_ended["xG"].values
        lr = LinearRegression()
        lr.fit(X2, y2)
        joblib.dump(lr, 'shotLR.pkl')

    except:
        print("Error with model building")

In [ ]:
def getPassingProbabilityPredictionFromModel(df):


In [ ]:
def getShotProbabilityPredictionFromModel(df,var):
    shot_ended = df.loc[df["shot_end"] == 1]
    X2 = shot_ended[var].values
    y_pred = lr.predict(X)
    df["xG_pred"] = y_pred

    passes = df.copy()
    return passes

In [6]:
def passingProbabilityPlots(df):

            #predict probability of shot ended
            y_pred_proba = model.predict_proba(X)[::,1]

            passes["shot_prob"] = y_pred_proba
            #OLS
            
            try:
                shot_ended = passes.loc[passes["shot_end"] == 1]
                X2 = shot_ended[var].values
                y2 = shot_ended["xG"].values
                lr = LinearRegression()
                lr.fit(X2, y2)
                y_pred = lr.predict(X)
                passes["xG_pred"] = y_pred
                #calculate xGchain
                passes["xT"] = passes["xG_pred"]*passes["shot_prob"]

                passes[["xG_pred", "shot_prob", "xT"]].head(5)
                plotPasseswithShotEnd(df3,unique_possessions,passes)
            except:
                print("Error with model building")
        else:
            st.write("Player had only one type of shots, can build a probability model if the dataset has only one type of class")
    else:
        st.write("Player had zero shots")


In [7]:
def plotPasseswithShotEnd(df,unique_possessions,passes):
    for i in unique_possessions:
        # plot possession chain that ended with shot
        chain = df.loc[df["possession"] == i]
        # get passes
        passes_in = passes.loc[df["possession"] == i]
        # get events different than pass
        not_pass = chain.loc[chain["type_name"] != "Pass"].iloc[:-1]
        # shot is the last event of the chain (or should be)
        shot = chain.iloc[-1]
        shot_taken_by = shot['player_name']
        shot_outcome = shot['outcome_name']
        if shot_outcome == 'Off T':
            shot_outcome = "Off Target shot"
        elif shot_outcome == 'Saved':
            shot_outcome = 'Saved Shot'
        elif shot_outcome == 'Blocked':
            shot_outcome = 'Blocked Shot'

        # plot
        pitch = Pitch(line_color='black', pitch_type='custom', pitch_length=120, pitch_width=80, line_zorder=2)
        fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                             endnote_height=0.04, title_space=0, endnote_space=0)
        # passes
        max_value = passes_in["xT"].max()  # Maximum xT value among passes
        pitch.arrows(passes_in.x0, passes_in.y0, passes_in.x1, passes_in.y1, color="blue", ax=ax['pitch'], zorder=3)
        
        # Annotate arrows with xT values
        for i, row in passes_in.iterrows():
            xT_value = row["xT"]
            arrow_x = (row.x0 + row.x1) / 2
            arrow_y = (row.y0 + row.y1) / 2
            size = 15 * (xT_value / max_value)  # Adjust the size based on xT value
            ax['pitch'].text(arrow_x, arrow_y, f"{xT_value:.3f}", color="Red", ha='center', va='center',fontsize = size)

        # shot
        pitch.arrows(shot.x0, shot.y0, shot.x1, shot.y1, color="red", ax=ax['pitch'], zorder=3)
        # other passes like arrows
        pitch.lines(not_pass.x0, not_pass.y0, not_pass.x1, not_pass.y1, color="grey", lw=1.5, ls='dotted', ax=ax['pitch'])
        ax['title'].text(0.5, 0.5, f'Passes leading to a {shot_outcome} taken by {shot_taken_by}', ha='center', va='center', fontsize=20)
        print("plottiung plot")
        st.pyplot(fig)


In [16]:
def preparePassingAndShotsModel(df):
    df_unique_possessions_chains = pd.DataFrame()
    for uniqueMatchID in df['match_id'].unique():
        df_final = df[df['match_id'] == uniqueMatchID]
        possessions = getPossessionsPerMatchDf(df_final)
        df_unique_possessions_chains = pd.concat([df_unique_possessions_chains, possessions], ignore_index=True)

        
    new_Df, var = featureEngineeringForModel(df_unique_possessions_chains)
    passingProbabilityModel(new_Df, var)
    print("model saved")

In [17]:
preparePassingAndShotsModel(df)

IndexError: index 0 is out of bounds for axis 0 with size 0